
# Documentação do Notebook: Limpeza de Dados de Parlamentares e Emendas Parlamentares

## Objetivo
Este notebook tem como objetivo realizar a limpeza e padronização das tabelas de parlamentares e emendas parlamentares.

## Etapas do Processo
1. **Exclusão de Colunas Irrelevantes**
    - Remoção de colunas que não são necessárias para a análise.

2. **Análise de Valores Faltantes**
    - Identificação e tratamento de valores nulos ou ausentes nas tabelas.

3. **Análise de Valores Inválidos**
    - Verificação e correção de dados inconsistentes ou fora do padrão esperado.

4. **Padronização de Case Sensitive**
    - Uniformização de textos para evitar discrepâncias causadas por diferenças de maiúsculas/minúsculas.

## Tabelas Utilizadas
  - **Parlamentares**
  - **Emendas Parlamentares**

## Resultado Esperado
Ao final do notebook, as tabelas estarão limpas, padronizadas e prontas para análises posteriores.

# Importação de bibliotecas e funções

In [0]:
from pyspark.sql import functions as F

# Tratamentos

## Tabela de emendas parlamentares

In [0]:
# Carregar a tabela de emendas parlamentares
emendas = spark.read.table("mvp.bronze.emendas")


In [0]:
# Visualizar as primeiras 5 linhas
emendas.limit(5).display()

In [0]:
# Colunas da tabela
emendas.columns

In [0]:
# Selecionar apenas as colunas desejadas
colunas_relevantes_emendas = ['Ano',
 'Autor_da_emenda',
 'Localidade_do_gasto__Regionalizacao_',
 'Funcao',
 'Valor_empenhado',
 'Valor_liquidado',
 'Valor_pago',
]
emendas_filtradas = emendas.select(colunas_relevantes_emendas)

emendas_filtradas.display()

### Análise da colunas

#### Ano

In [0]:
emendas_filtradas.select("Ano").printSchema()

A coluna de ano está como string e precisará ser alterada para inteiro

In [0]:
emendas_filtradas = emendas_filtradas.withColumn("Ano", F.col("Ano").cast("int"))
emendas_filtradas.select("Ano").printSchema()



In [0]:
emendas_filtradas.filter(F.col("Ano").isNull()).count()

Não temos valores nulos para a coluna de Ano

In [0]:
emendas_filtradas.groupBy("Ano").count().orderBy(F.desc("count")).display()

Não temos vazios temporais para os anos considerados. O número de emendas de 2025 é menor, mas é próximo ao número de emendas de 2023 e ainda estamos no ano corrente de 2025, possibilitando ainda novas emendas.

#### Autor da Emenda

In [0]:
emendas_filtradas.select("Autor_da_emenda").printSchema()

O Formato está correto para essa coluna

In [0]:
emendas_filtradas.filter(F.col("Autor_da_emenda").isNull()).count()

Não temos valores nulos para autores de emendas

In [0]:
emendas_filtradas.limit(5).display()

Foi notado que temos autores de emendas que não são parlamentares, mas comissões. Vamos investigar mais afundo

In [0]:
emendas_filtradas.groupBy("Autor_da_emenda").count().orderBy(F.desc("count")).display()

Vão ser eliminadas da base de autores:
- Bancadas (BANCADA)
- Comissões (COM.)

Além disso, foi notado que o número de parlamentares contidos na base de emendas está bem superior ao valor obtido na base de parlamentares e vamos precisar investigar melhor qual será o tratamento.


In [0]:
emendas_filtradas = emendas_filtradas.filter(
    ~(
        F.col("Autor_da_emenda").like("%BANCADA%") |
        F.col("Autor_da_emenda").like("%COM.%")
    )
)


In [0]:
emendas_filtradas.groupBy("Autor_da_emenda").count().orderBy(F.asc("count")).limit(10).display()

Ao buscar nomes como Vinicius Poit, Beto Faro e Marcelo Nilo, observamos que são deputados federais do mandato anterior, cujas emendas foram executadas no ano seguinte, já na nova legislatura. Para nossas análises, esses parlamentares serão desconsiderados, pois não constam na base atual de parlamentares.

In [0]:
emendas_filtradas = emendas_filtradas.withColumn(
    "nome_parlamentar",
    F.upper(
        F.trim(
            F.regexp_extract(
                F.col("Autor_da_emenda"),
                r"-\s*(.*?)(?=\(|$)",
                1
            )
        )
    )
)

A coluna "nome_parlamentar" foi criada extraindo o nome após o caractere "-" até o final do texto, ignorando qualquer conteúdo entre parênteses. O resultado foi convertido para maiúsculas e espaços em branco foram removidos.

In [0]:
emendas_filtradas.groupBy("nome_parlamentar").count().orderBy(F.desc("count")).display()

In [0]:
emendas_filtradas = emendas_filtradas.drop("Autor_da_emenda")

In [0]:
emendas_filtradas.display()

#### Localidade

In [0]:
emendas_filtradas.select("Localidade_do_gasto__Regionalizacao_").printSchema()

A Coluna está como string e é o tipo correto para a coluna

In [0]:
emendas_filtradas.filter(F.col("Localidade_do_gasto__Regionalizacao_").isNull()).count()

Não temos valores nulos para localidade da emenda parlamentar

In [0]:
emendas_filtradas.groupBy("Localidade_do_gasto__Regionalizacao_").count().orderBy(F.desc("count")).display()

Ao analisar os valores, foram percebidos três tipos de informações:
- Emendas associadas a cidades
- Emendas associados a estados
- Emendas associadas a regiões, nacionais ou múltiplas

Por isso será realizado o seguinte tratamento:

O tratamento da coluna de localidade padroniza todo o texto para uppercase e remove espaços extras. Quando a localidade está no formato “MUNICÍPIO - UF”, o município é extraído e a UF final é identificada diretamente. Caso contrário, o código tenta identificar uma UF no final da string; se não existir, extrai o nome do estado e utiliza um dicionário de mapeamento para obter a sigla correspondente. A coluna uf_destino segue essa ordem de prioridade: primeiro a UF do padrão de município, depois a UF no final da string e, por fim, a UF obtida via mapeamento do estado.

In [0]:
# Dicionário com o mapeamento de estados para siglas
uf_map = {
    "ACRE": "AC","ALAGOAS": "AL","AMAPÁ": "AP","AMAPA": "AP","AMAZONAS": "AM",
    "BAHIA": "BA","CEARÁ": "CE","CEARA": "CE","DISTRITO FEDERAL": "DF",
    "ESPÍRITO SANTO": "ES","ESPIRITO SANTO": "ES","GOIÁS": "GO","GOIAS": "GO",
    "MARANHÃO": "MA","MARANHAO": "MA","MATO GROSSO": "MT","MATO GROSSO DO SUL": "MS",
    "MINAS GERAIS": "MG","PARÁ": "PA","PARA": "PA","PARAÍBA": "PB","PARAIBA": "PB",
    "PARANÁ": "PR","PARANA": "PR","PERNAMBUCO": "PE","PIAUÍ": "PI","PIAUI": "PI",
    "RIO DE JANEIRO": "RJ","RIO GRANDE DO NORTE": "RN","RIO GRANDE DO SUL": "RS",
    "RONDÔNIA": "RO","RONDONIA": "RO","RORAIMA": "RR","SANTA CATARINA": "SC",
    "SÃO PAULO": "SP","SAO PAULO": "SP","SERGIPE": "SE","TOCANTINS": "TO",
}

mapping_expr = F.create_map([F.lit(x) for pair in uf_map.items() for x in pair])


emendas_filtradas = emendas_filtradas.withColumn("localidade_raw", F.upper(F.col("Localidade_do_gasto__Regionalizacao_").cast("string")))

emendas_filtradas = (
    emendas_filtradas
    # MUNICÍPIO quando tiver " - "
    .withColumn(
        "municipio_extr",
        F.when(
            F.col("localidade_raw").contains(" - "),
            F.trim(F.regexp_extract(F.col("localidade_raw"), r"^(.*?)-", 1))
        )
    )

    # UF encontrada pelo final da string (ex: " - RJ")
    .withColumn(
        "uf_final",
        F.regexp_extract(F.col("localidade_raw"), r"\b([A-Z]{2})$", 1)
    )

    # Nome do estado (para mapear)
    .withColumn(
        "estado_extr",
        F.trim(F.regexp_extract(F.col("localidade_raw"), r"^([A-ZÀ-Ü ]+)", 1))
    )


    # LOCALIDADE SEMPRE UPPERCASE
    .withColumn("localidade_destino", F.col("localidade_raw"))

    # MUNICÍPIO DESTINO
    .withColumn(
        "municipio_destino",
        F.col("municipio_extr")
    )

    # LÓGICA FINAL DA UF
    .withColumn(
        "uf_destino",
        F.when(F.col("municipio_extr").isNotNull(), F.col("uf_final"))               # caso "CIDADE - UF"
         .when(F.col("uf_final") != "", F.col("uf_final"))                          # final = UF
         .otherwise(mapping_expr[F.col("estado_extr")])                              # nome → UF (mapeamento)
    )

    .drop("Localidade_do_gasto__Regionalizacao_","localidade_raw", "municipio_extr", "estado_extr", "uf_final")
)

In [0]:
emendas_filtradas\
    .filter(F.col("uf_destino").isNull())\
    .groupBy("localidade_destino")\
    .count()\
    .orderBy(F.desc("count"))\
    .display()


A Lógica está funcionando corretamente, só temos valores vazios quando não temos um estado definido.

#### Função

In [0]:
emendas_filtradas.select("Funcao").printSchema()

A coluna de função está como string que é tipo correto da coluna

In [0]:
emendas_filtradas.filter(F.col("Funcao").isNull()).count()

Não temos valores nulos para coluna de função

In [0]:
emendas_filtradas.groupBy("Funcao").count().orderBy(F.desc("count")).display()

Todos os 27 tipos de emendas parlamentares são válidos

#### Valores

In [0]:
emendas_filtradas.select("Valor_empenhado", "Valor_liquidado", "Valor_pago").printSchema()

Os valores estão com os formatos incorretos e precisarão ser corrigidos.

In [0]:
emendas_filtradas = emendas_filtradas.withColumn(
    "Valor_empenhado",
    F.regexp_replace(
        F.regexp_replace(F.col("Valor_empenhado"), r"\.", ""),
        ",", "."
    ).cast("double")
)

emendas_filtradas = emendas_filtradas.withColumn(
    "Valor_liquidado",
    F.regexp_replace(
        F.regexp_replace(F.col("Valor_liquidado"), r"\.", ""),
        ",", "."
    ).cast("double")
)

emendas_filtradas = emendas_filtradas.withColumn(
    "Valor_pago",
    F.regexp_replace(
        F.regexp_replace(F.col("Valor_pago"), r"\.", ""),
        ",", "."
    ).cast("double")
)

In [0]:
emendas_filtradas.limit(10).display()

Para validar as colunas de valores, não podemos ter valores nulos. <Br>
Além disso, o valor empenhado não pode ser menor que o valor liquidado e o valor liquidado não pode ser menor que o valor pago.

In [0]:
colunas_valores = ["Valor_empenhado", "Valor_liquidado", "Valor_pago"]

In [0]:
emendas_filtradas.select([
    F.col(c).isNull().cast("int").alias(c)
    for c in colunas_valores
]).groupBy().sum().show()

Não temos valores nulos para as colunas de valores

In [0]:

emendas_filtradas.select(colunas_valores).describe().display()

Toda emenda contém ao menos um valor empenhado

In [0]:
emendas_filtradas.select(F.max("Valor_empenhado")).show()

In [0]:
emendas_filtradas.orderBy(F.desc("valor_empenhado")).limit(10).display()

Temos emendas parlamentares de 68 milhões de reais com esses valores pagos

In [0]:
df_invalidos = emendas_filtradas.filter(
    (F.col("Valor_empenhado") < F.col("Valor_liquidado")) |
    (F.col("Valor_liquidado") < F.col("Valor_pago"))
)

In [0]:
df_invalidos.display()

Podemos notar 96 emendas que não respeitam a regra de o valor ser empenhado (Dedicado a um tema), liquidado (Designado a pagamento) e pago (Quando é efetivamente realizado a transfêrencia de dinheiro).

In [0]:
df_invalidos_2 = emendas_filtradas.filter(
    (F.col("Valor_empenhado") < F.col("Valor_liquidado")) 
)

In [0]:
df_invalidos_2.display()

Temos 55 emendas com valores empenhados menores do que os valores liquidados. Ou seja, foram gastos mais do que planejado.

### Salvar tabela

In [0]:
# Renomear colunas e agregar emendas com as mesmas informações
emendas_silver = (
    emendas_filtradas
    .select(
        F.col("nome_parlamentar"),
        F.col("Ano").alias("ano"),
        F.col("Funcao").alias("funcao"),
        F.col("Valor_empenhado").alias("valor_empenhado"),
        F.col("Valor_liquidado").alias("valor_liquidado"),
        F.col("Valor_pago").alias("valor_pago"),
        F.col("localidade_destino").alias("localidade_destino"),
        F.col("municipio_destino").alias("municipio_destino"),
        F.col("uf_destino").alias("uf_destino")
    )
    .groupBy(
        "nome_parlamentar",
        "ano",
        "funcao",
        "localidade_destino",
        "municipio_destino",
        "uf_destino"
    )
    .agg(
        F.sum("valor_empenhado").alias("valor_empenhado"),
        F.sum("valor_liquidado").alias("valor_liquidado"),
        F.sum("valor_pago").alias("valor_pago")
    )
)

In [0]:
emendas_silver.display()

In [0]:
emendas_silver.count()

Temos agora 12374 emendas parlamentares após a remoção de emendas de bancadas, comissões e agregação de emendas com mesmos metadados

In [0]:
emendas_silver.write.format("delta").mode("overwrite").saveAsTable("mvp.silver.emendas")

In [0]:
spark.sql("""
COMMENT ON TABLE mvp.silver.emendas IS 'Tabela agregada de emendas parlamentares, contendo valores empenhados, liquidados e pagos por parlamentar, ano, função e destino.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.silver.emendas.nome_parlamentar IS 'Nome padronizado do parlamentar autor da emenda.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.silver.emendas.ano IS 'Ano de referência da emenda parlamentar. (Valores válidos: 2023, 2024, 2025)'
""")

spark.sql("""
COMMENT ON COLUMN mvp.silver.emendas.funcao IS 'Função orçamentária associada à emenda.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.silver.emendas.localidade_destino IS 'Localidade de destino dos recursos da emenda.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.silver.emendas.municipio_destino IS 'Município de destino dos recursos da emenda.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.silver.emendas.uf_destino IS 'Unidade Federativa (UF) de destino dos recursos da emenda.'
""")

spark.sql("""
COMMENT ON COLUMN mvp.silver.emendas.valor_empenhado IS 'Valor total empenhado na emenda parlamentar. (Valores maiores que 0)'
""")

spark.sql("""
COMMENT ON COLUMN mvp.silver.emendas.valor_liquidado IS 'Valor total liquidado na emenda parlamentar. (Valores maiores ou iguais a 0)'
""")

spark.sql("""
COMMENT ON COLUMN mvp.silver.emendas.valor_pago IS 'Valor total pago na emenda parlamentar.(Valores maiores ou iguais a 0)'
""")

## Tabela de parlamentares

In [0]:
# Carregar a tabela de parlamentares
parlamentares = spark.read.table("mvp.bronze.parlamentares")

In [0]:
# Visualizar as primeiras 5 linhas
parlamentares.limit(5).display()

In [0]:
# Colunas da tabela
parlamentares.columns

In [0]:
# Selecionar apenas as colunas desejadas
colunas_relevantes_parlamentares = ['nome',
 'siglaPartido',
 'siglaUf'
]
parlamentares_filtrados = parlamentares.select(colunas_relevantes_parlamentares)

parlamentares_filtrados.display()

### Análise de duplicidade

In [0]:
chave = [
    "nome",
    "siglaPartido",
    "siglaUF"
]

duplicados = (
    parlamentares_filtrados
    .groupBy(chave)
    .count()
    .filter(F.col("count") > 1)
)

duplicados.count()

Não temos parlamentares duplicados

### Análise de nulos

In [0]:
# Contagem de nulos para cada coluna
nulos = parlamentares_filtrados.select([
    F.sum(F.col(c).isNull().cast("int")).alias(c) for c in colunas_relevantes_parlamentares
])

nulos.show()

Não temos nulos na colunas da tabela de parlamentares

### Análise das colunas

#### Nome do Parlamentar

Para as colunas de nome precisamos realizar uma padronização. Por isso, precisamos remover acentos e utilizar o upper case

In [0]:
def padroniza_nome(coluna):
    return F.upper(
        F.translate(
            F.col(coluna),
            "áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ",
            "aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC"
        )
    )

In [0]:
parlamentares_filtrados = parlamentares_filtrados.withColumn("nome_parlamentar", padroniza_nome("nome"))

In [0]:
parlamentares_filtrados.groupBy("nome_parlamentar").count().orderBy(F.desc("count")).display()

#### Sigla do Partido

In [0]:
parlamentares_filtrados.groupBy("siglaPartido").count().orderBy(F.desc("count")).display()

Podemos ver 20 partidos dentro da base de parlamentares e todos são válidos

#### Sigla da UF

In [0]:
parlamentares_filtrados.groupBy("siglaUf").count().orderBy(F.desc("count")).display()

Podemos ver as 27 unidades federativas e todas com ao menos 8 deputados federais

### Salvar tabela

In [0]:
parlamentares_filtrados.display()

In [0]:
# Renomear colunas e agregar emendas com as mesmas informações
parlamentares_silver = parlamentares_filtrados.select("nome_parlamentar", "siglaPartido", "siglaUf")

In [0]:
parlamentares_silver.display()

In [0]:
parlamentares_silver.count()

In [0]:
parlamentares_silver.write.format("delta").mode("overwrite").saveAsTable("mvp.silver.parlamentares")

In [0]:
spark.sql("""
  COMMENT ON TABLE mvp.silver.parlamentares IS 'Tabela silver contendo parlamentares, com nome, partido e UF';
""")

spark.sql("""
  COMMENT ON COLUMN mvp.silver.parlamentares.nome_parlamentar IS 'Nome do parlamentar';
""")

spark.sql("""
  COMMENT ON COLUMN mvp.silver.parlamentares.siglaPartido IS 'Sigla do partido do parlamentar';
""")

spark.sql("""
  COMMENT ON COLUMN mvp.silver.parlamentares.siglaUf IS 'Sigla da unidade federativa do parlamentar';
""")